<a href="https://colab.research.google.com/github/priyanshunayak05/NLP/blob/main/Assignment1_BERT(Pretrained_model)_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers # only run this once per kernel session - dont want to overload kaggle

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import transformers as ppb # BERT Model
import tensorflow as tf
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import regexp_tokenize
from sklearn.model_selection import train_test_split


# Load your CSV file (make sure it's uploaded in Colab or mounted from Drive)
df = pd.read_csv("/content/funny_texts.csv")
# If your dataset columns are named `text` and `label`, this works directly:
X = df['text']
y = df['label']

# Split into train/test sets (same format as Kaggle version)
train_x, test_x, train_y, test_y = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training samples:", len(train_x))
print("Testing samples:", len(test_x))

Training samples: 48
Testing samples: 13


**Why we use HuggingFace?**

HuggingFace is like a **library of pre-trained AI model**s.

It provides ready-made models (like BERT, GPT, RoBERTa, T5, etc.).

It also provides tokenizers (to convert text → tokens).

In [8]:
!rm -rf ~/.cache/huggingface/

**🤖 Why we use PyTorch or TensorFlow then?**

HuggingFace provides the models,
But the models must run on a deep learning framework.


HuggingFace gives the models,

PyTorch/TensorFlow run the models.

In [10]:
import transformers
import tensorflow
print(f"Transformers version: {transformers.__version__}")
print(f"TensorFlow version: {tensorflow.__version__}")

Transformers version: 4.57.1
TensorFlow version: 2.19.0


BERT stands for **Bidirectional Encoder Representations from Transformers**.

It is a pre-trained deep learning language model created by Google.

In simple words:

BERT is a ready-made brain that already understands English, because it has been trained on millions of sentences (Wikipedia + Books).


BERT is based on the Transformer Encoder

🏋️‍♂️ Training Style

BERT was pre-trained using two tasks:

Task	Meaning

**Masked Language Modeling (MLM**)	  Random words are masked and BERT predicts them

**Next Sentence Prediction (NSP)**	Predicts if one sentence logically follows another

This makes BERT very good at understanding context and relationships in text.

🧠 Why is BERT special?

Previous models understood text **left → right** (one direction) or **right → left.**

But **BERT** reads the text in **both directions** at the same time (bidirectional).

Example sentence:

"He went to the bank to deposit money."

Here bank means financial institution, not river bank.
BERT understands that because it looks at the words before and after it.

Below is the main model and tokenizer setup.

To tokenize our data, we use huggingface's BertTokenizer, which does some extra stuff on top of our own cleaning, like adding tokens like [CLS], and other necessary steps for BERT. We still have to do the same data cleaning that we had done previously, and so I copied over the steps from there into here.

The model is the TFBertForSequenceClassification model, which is basically a seqeunce classifier (like we want). This is better than previous models since the classification step and fine-tuning step are packed into one step, making it easier for us to use and work with.

In [12]:
# Attempt to load the model explicitly for TensorFlow and disable safetensors
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", from_pt=True, use_safetensors=False)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
tokenizer = ppb.AutoTokenizer.from_pretrained("bert-base-uncased")

These data cleaning functions below perform the same functions as were done in the main data processing notebook:

lemmatize() - lemmatizes the sentence input s, helps simplify model vocabulary

lower() - lowercases all the words in the sentence inputs s - is a remnant of a previous iteration, but I kept it around since it did no harm

clean() - a generalized cleaning function that does the two steps above + removes all numbers from data, list of sentences passed in

tokenize() - tokenizes the list of sentences passed in text - this is what the BertTokenizer from the transformers library does. Returns an array of word vectors.

process() - a combination of cleaning and tokenizing, a function really created for our ease of use

In [21]:
def lemmatize(s):
    wordnet_lemmatizer = WordNetLemmatizer()
    return " ".join([wordnet_lemmatizer.lemmatize(w,'v') for w in s.split(" ")])

def lower(s):
    return s.lower()

def clean(data):
    # Apply cleaning steps to each element in the pandas Series
    data = data.apply(lambda x: lemmatize(x))
    data = data.apply(lambda x: lower(x))
    data = data.apply(lambda x: re.sub(r'\d+', '', x)) # remove nums
    return data

def tokenize(text):
    # Ensure text is a list of strings for batch processing
    if isinstance(text, pd.Series):
        text = text.tolist()
    tokenized = tokenizer(text, padding=True, truncation=True, return_tensors="tf")
    return tokenized

def process(data):
    cleaned = clean(data)
    return tokenize(cleaned)

In [22]:
train_batch = process(train_x)
test_batch = process(test_x)

In [19]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [23]:
learning_rate = 2e-5
epochs = 10
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-8)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric1 = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
metric2 = tf.keras.metrics.Precision(name="precision")
metric3 = tf.keras.metrics.Recall(name="recall")
model.compile(optimizer=optimizer, loss=loss, metrics=[metric1])

In [24]:
history = model.fit(x=train_batch.input_ids, y=np.array(train_y), epochs=epochs)

Epoch 1/10
2/2 [==============================] - 66s 7s/step - loss: 0.6953 - accuracy: 0.5208
Epoch 2/10
2/2 [==============================] - 20s 7s/step - loss: 0.6205 - accuracy: 0.8542
Epoch 3/10
2/2 [==============================] - 20s 8s/step - loss: 0.5492 - accuracy: 0.9583
Epoch 4/10
2/2 [==============================] - 20s 7s/step - loss: 0.4804 - accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 21s 8s/step - loss: 0.3836 - accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 20s 6s/step - loss: 0.3014 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 21s 8s/step - loss: 0.2293 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - 20s 7s/step - loss: 0.1943 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 26s 8s/step - loss: 0.1574 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 31s 12s/step - loss: 0.1242 - accuracy: 1.0000


In [25]:
model.evaluate(x=test_batch.input_ids, y=np.array(test_y))

1/1 [==============================] - 7s 7s/step - loss: 0.0963 - accuracy: 1.0000


[0.0963495671749115, 1.0]

In [26]:
from sklearn.metrics import classification_report
y_pred = model.predict(x=test_batch.input_ids)


1/1 [==============================] - 5s 5s/step


In [31]:
print(y_pred[0].shape)  # 13 data point or 2 class hai  funny or not funny
y_pred_bool = np.argmax(y_pred[0], axis=1) # sabse badha score jada hai
print(np.array(test_y).shape)
print(y_pred_bool)
print(y_pred[0])
print(classification_report(test_y, y_pred_bool,))

# Macro average: Har class ka score lekar unka saadharan average nikaalna, yaani sabhi class ko barabar importance dena.

# Weighted average: Har class ke score ko us class ke samples ki sankhya ke hisaab se weight dekar average nikaalna, taaki badi classes ka asar zyada dikhe.


(13, 2)
(13,)
[0 1 1 0 1 0 1 0 0 1 0 1 0]
[[ 1.368714   -1.5068904 ]
 [-0.86342055  1.0902638 ]
 [-0.94209534  1.1208786 ]
 [ 1.2954127  -1.5229827 ]
 [-0.8851504   1.1726055 ]
 [ 1.3456918  -1.5748606 ]
 [-0.86865395  1.1251256 ]
 [ 1.1863793  -1.2852141 ]
 [ 1.2930834  -1.2206954 ]
 [-0.8985639   1.1045573 ]
 [ 1.3265722  -1.654271  ]
 [-0.7067116   0.8390309 ]
 [ 1.2472755  -1.5546747 ]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00         6

    accuracy                           1.00        13
   macro avg       1.00      1.00      1.00        13
weighted avg       1.00      1.00      1.00        13



**model.predict()** returns logits (raw model output scores) representing the probability of each class.

It converts text into token IDs that your model can understand.

Output contains:

**input_ids** → numerical tokens for each word/piece

**attention_mask** → tells the model which tokens are real (1) and which are padding (0)

**token_type_ids** → distinguishes sentence pairs (useful for tasks like question-answering), but still included here.

In [32]:
tk = tokenizer("When my son told me to stop impersonating a flamingo, I had to put my foot down.", padding=True, return_tensors="tf")
out = model.predict(x=(tk.input_ids, tk.attention_mask, tk.token_type_ids))
print(np.argmax(out[0], axis=1))

1/1 [==============================] - 0s 252ms/step
[1]


In [30]:
submission = pd.DataFrame({"Prediction":y_pred_bool})
submission.to_csv("predictions.csv", index=True, index_label="Id")